In [1]:
# 1. import libraries
from bs4 import BeautifulSoup
import requests
import pandas as pd

In [2]:
google = requests.get('https://developers.google.com')
print('Google:', google.status_code)

Google: 200


In [3]:
NBA = response = requests.get('https://api.sportsdata.io/api/nba/fantasy/json/CurrentSeason')
print('NBA:', NBA.status_code)

NBA: 401


In [4]:
rotten_tomato = requests.get('http://api.rottentomatoes.com/api/public/v1.0/lists/movies/box_office.json')
print('Rotten Tomatoes:', rotten_tomato.status_code)

Rotten Tomatoes: 403


In [5]:
# flights from where to where on what date
url = 'https://skyscanner-skyscanner-flight-search-v1.p.rapidapi.com/apiservices/browsequotes/v1.0/UK/GBP/en-GB/SFO-sky/NYCA-sky/2020-12-12'
params = {'inboundpartialdate':'2020-12-12'}
headers = {
    'x-rapidapi-host': 'skyscanner-skyscanner-flight-search-v1.p.rapidapi.com',
    'x-rapidapi-key': 'replacewithyourkey'}
response = requests.get(url, headers=headers, params=params)
response.json()

{'message': 'You are not subscribed to this API.'}

In [6]:
#this wasn't really asked for in the lab, but i decided to give it a go at the spotify api instead of doing flights again (too depressing at the moment)


In [7]:
import spotipy
import spotipy.util as util
import spotipy.oauth2 as oauth2

In [8]:
CLIENT_ID = "a0c672c0bbdd45e59db228728d00b779"
CLIENT_SECRET = "28771ac72d194f758a908beedd093358"

In [9]:
token = oauth2.SpotifyClientCredentials(client_id=CLIENT_ID, client_secret=CLIENT_SECRET)
cache_token = token.get_access_token()
sp = spotipy.Spotify(cache_token)

<ipython-input-9-5c2b7eb2a594>:2: DeprecationWarning: You're using 'as_dict = True'.get_access_token will return the token string directly in future versions. Please adjust your code accordingly, or use get_cached_token instead.
  cache_token = token.get_access_token()


In [10]:
sp.user_playlist_tracks("Epidemic Sound", "7zLyqrGV0EoJUGwCG5WKew")

{'href': 'https://api.spotify.com/v1/playlists/7zLyqrGV0EoJUGwCG5WKew/tracks?offset=0&limit=100&additional_types=track',
 'items': [{'added_at': '2020-11-30T08:22:54Z',
   'added_by': {'external_urls': {'spotify': 'https://open.spotify.com/user/epidemicsound'},
    'href': 'https://api.spotify.com/v1/users/epidemicsound',
    'id': 'epidemicsound',
    'type': 'user',
    'uri': 'spotify:user:epidemicsound'},
   'is_local': False,
   'primary_color': None,
   'track': {'album': {'album_type': 'single',
     'artists': [{'external_urls': {'spotify': 'https://open.spotify.com/artist/2TTmUTfsQjQb9Wx8arAZy9'},
       'href': 'https://api.spotify.com/v1/artists/2TTmUTfsQjQb9Wx8arAZy9',
       'id': '2TTmUTfsQjQb9Wx8arAZy9',
       'name': 'Tyra Chantey',
       'type': 'artist',
       'uri': 'spotify:artist:2TTmUTfsQjQb9Wx8arAZy9'}],
     'available_markets': ['AD',
      'AE',
      'AL',
      'AR',
      'AT',
      'AU',
      'BA',
      'BE',
      'BG',
      'BH',
      'BO',
     

In [11]:
playlist = sp.user_playlist_tracks("Epidemic Sound", "7zLyqrGV0EoJUGwCG5WKew")

In [12]:
playlist2 = sp.user_playlist_tracks("Epidemic Sound", "7zLyqrGV0EoJUGwCG5WKew")["items"]

In [13]:
playlist2

[{'added_at': '2020-11-30T08:22:54Z',
  'added_by': {'external_urls': {'spotify': 'https://open.spotify.com/user/epidemicsound'},
   'href': 'https://api.spotify.com/v1/users/epidemicsound',
   'id': 'epidemicsound',
   'type': 'user',
   'uri': 'spotify:user:epidemicsound'},
  'is_local': False,
  'primary_color': None,
  'track': {'album': {'album_type': 'single',
    'artists': [{'external_urls': {'spotify': 'https://open.spotify.com/artist/2TTmUTfsQjQb9Wx8arAZy9'},
      'href': 'https://api.spotify.com/v1/artists/2TTmUTfsQjQb9Wx8arAZy9',
      'id': '2TTmUTfsQjQb9Wx8arAZy9',
      'name': 'Tyra Chantey',
      'type': 'artist',
      'uri': 'spotify:artist:2TTmUTfsQjQb9Wx8arAZy9'}],
    'available_markets': ['AD',
     'AE',
     'AL',
     'AR',
     'AT',
     'AU',
     'BA',
     'BE',
     'BG',
     'BH',
     'BO',
     'BR',
     'BY',
     'CA',
     'CH',
     'CL',
     'CO',
     'CR',
     'CY',
     'CZ',
     'DE',
     'DK',
     'DO',
     'DZ',
     'EC',
     'E

In [21]:
def analyze_playlist(creator, playlist_id):
    
    # Create empty dataframe
    playlist_features_list = ["artist","album","track_name",  "track_id","danceability","energy","key","loudness","mode", "speechiness","instrumentalness","liveness","valence","tempo", "duration_ms","time_signature"]
    
    playlist_df = pd.DataFrame(columns = playlist_features_list)
    
    # Loop through every track in the playlist, extract features and append the features to the playlist df
    
    playlist = sp.user_playlist_tracks(creator, playlist_id)["items"]
    for track in playlist:
        # Create empty dict
        playlist_features = {}
        # Get metadata
        playlist_features["artist"] = track["track"]["album"]["artists"][0]["name"]
        playlist_features["album"] = track["track"]["album"]["name"]
        playlist_features["track_name"] = track["track"]["name"]
        playlist_features["track_id"] = track["track"]["id"]
        
        # Get audio features
        audio_features = sp.audio_features(playlist_features["track_id"])[0]
        for feature in playlist_features_list[4:]:
            playlist_features[feature] = audio_features[feature]
        
        # Concat the dfs
        track_df = pd.DataFrame(playlist_features, index = [0])
        playlist_df = pd.concat([playlist_df, track_df], ignore_index = True)
        
    return playlist_df
        

In [22]:
analyze_playlist("Epidemic Sound", "7zLyqrGV0EoJUGwCG5WKew")

,artist,album,track_name,track_id,danceability,energy,key,loudness,mode,speechiness,instrumentalness,liveness,valence,tempo,duration_ms,time_signature
0,Tyra Chantey,Sweet Talk,Sweet Talk,3smc46gHnjibhEqkYFoRKO,0.805,0.625,11,-4.626,1,0.0405,0,0.0583,0.702,110.012,168000,4
1,King Sis,Yeah You (Thinkin About You),Yeah You (Thinkin Bout You),4DdvK2sYroCbmttW2b0VvC,0.776,0.501,4,-7.396,0,0.1730,0,0.2610,0.509,101.940,183695,4
2,Ella Faye,I Was Messed Up,I Was Messed Up,0RvaInrXZneP1452L3ftI2,0.716,0.575,0,-8.878,1,0.0301,0.000602,0.1410,0.493,104.998,185280,4
3,Loving Caliber,Dive Into You,Dive Into You,6cBP5oNeTYTdgEAikrS5nv,0.497,0.478,5,-7.801,1,0.0627,0,0.4900,0.395,85.392,204224,4
4,Deanz,Don't Give A Damn,Don’t Give a Damn,0zhI03BXtCBLBLJaPH0E4Q,0.778,0.487,9,-6.845,0,0.0490,0.0765,0.0863,0.571,119.205,167225,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,Claudia Kelley,Don't Wanna Go Deep,Don't Wanna Go Deep,2G9YriPDxkBCLFYWWQIZ1w,0.665,0.738,10,-7.219,0,0.2230,0.000157,0.2610,0.412,105.089,200254,4
96,Tommy Ljungberg,White Gold,White Gold,6A0seBGDXeogIDxS6P1e0r,0.519,0.826,0,-3.502,0,0.0536,1.99e-05,0.1730,0.667,169.987,184456,4
97,Snake City,Hurts,What You Saying,0IK4y7ClzPBEJ0jRCDUH13,0.735,0.816,3,-7.830,0,0.1270,0.0344,0.1110,0.303,115.974,219020,4
98,Ingrid Witt,Fear Being Unfelt,Fear Being Unfelt,3WfNoGOoR1jfO6brOOurjo,0.672,0.586,9,-13.270,0,0.2110,0.44,0.0744,0.283,180.010,186568,4


In [ ]:
#my variation of the code does not work, need to check why

In [15]:
def analyze_playlist2(creator, playlist_id):
    
    # Create empty dataframe
    playlist_features_list = ["artist","track_name"]
    playlist_df = pd.DataFrame(columns = playlist_features_list)
    
    
    # Loop through every track in the playlist, extract features and append the features to the playlist df
    
    playlist2 = sp.user_playlist_tracks(creator, playlist_id)["items"]
    
    
    for track in playlist2:
        # Create empty dict
        playlist_features = {}
        
        # Get metadata
        playlist_features["artist"] = track["track"]["album"]["artists"][0]["name"]
        playlist_features["track_name"] = track["track"]["name"]
        
    #add to dataframe
        playlist_df = pd.DataFrame(playlist_features, index = [0])
        
        
    return playlist_df